Symbolic operations
-------------------
First, let us bypass the debate over Python 2 and 3 by forcing us to write code that functions identically in either version:

In [1]:
from __future__ import print_function, division

We also want nicely formatted output:

In [2]:
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

Then we bulk-import everything we might need:

In [3]:
from sympy import *

We define some symbols:

In [4]:
x, y = symbols('x y')

In [5]:
sin(pi*x)/cos(pi*y)

sin(π⋅x)
────────
cos(π⋅y)

It renders nicely as $\frac{\sin\pi x}{\cos\pi x}$. Now a symbolic integral:

In [6]:
integrate(pi*sin(x*y), x)

  ⎛⎧    0       for y = 0⎞
  ⎜⎪                     ⎟
π⋅⎜⎨-cos(x⋅y)            ⎟
  ⎜⎪──────────  otherwise⎟
  ⎝⎩    y                ⎠

Evaluate this for a particular value of x and y:

In [9]:
integrate(pi*sin(x*y), x).subs([(x, pi), (y, 1)])

π

SymPy, just like Mathematica, keeps results symbolic as long as possible. If you want a numerical result, you specifically have to request it:

In [25]:
N(integrate(pi*sin(x*y), x).subs([(x, pi), (y, 1)]))

3.14159265358979

The quantum physics module is especially helpful. The noncommutative algebra is better than the respective package in Mathematica, especially when it comes to non-Hermitian variables:

In [7]:
from sympy.physics.quantum import *
X = HermitianOperator('X')
Y = Operator('Y')
Dagger(X*Y)

 †  
Y ⋅X

We can easily define Hamiltonians. For instance, the Hubbard model on a chain is as follows:

In [31]:
t = 1.0
U = 4.0
n_sites = 2
cu = [Operator("%s_%s_u" % ("c", i + 1)) for i in range(n_sites)]
cd = [Operator("%s_%s_d" % ("c", i + 1)) for i in range(n_sites)]
hamiltonian = sum(U*Dagger(cu[r])*cu[r]*Dagger(cd[r])*cd[r] for r in range(n_sites))
hamiltonian += sum(-t*(Dagger(cu[r])*cu[r+1]+Dagger(cu[r+1])*cu[r]
                       +Dagger(cd[r])*cd[r+1]+Dagger(cd[r+1])*cd[r]) for r in range(n_sites-1))
expand(hamiltonian)

              †                 †           †                    †            
- - -1.0⋅c_1_d ⋅c_2_d + 4.0⋅c₁ ᵤ ⋅c₁ ᵤ⋅c_1_d ⋅c_1_d - - -1.0⋅c₁ ᵤ ⋅c₂ ᵤ - - -1

        †                    †                †           †      
.0⋅c_2_d ⋅c_1_d - - -1.0⋅c₂ ᵤ ⋅c₁ ᵤ + 4.0⋅c₂ ᵤ ⋅c₂ ᵤ⋅c_2_d ⋅c_2_d

Functional programming
----------------------

Python was retrofitted with some elements of functional programming. It nevertheless remains an object-oriented language, but it is highly opportunistic. This approach is a lot like Mathematica, which is quintessentially functional, but you can follow any programming paradigm when using it.

List comprehensions are probably the most used construct in Python from functional programming. In fact, it is considered a Pythonesque way of doing things. It is a quick way of generating transformed lists from other lists: list comprehension is a simple map function in disguise.

In [22]:
[i**2 for i in range(5)]

[0, 1, 4, 9, 16]

The variable inside the list comprehension is similar to a pure function and we can have arbitrary number of them:

In [26]:
[i*j for i in range(1, 4) for j in range(1, 4)]

[1, 2, 3, 2, 4, 6, 3, 6, 9]

We can also include conditionals in the list comprehension:

In [27]:
[i for i in range(30) if i%3 == 0]

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

It is easy to emulate ``MapIndex``:

In [29]:
[[a, i] for i, a in enumerate([sqrt(2), pi, x])]

⎡⎡  ___   ⎤                ⎤
⎣⎣╲╱ 2 , 0⎦, [π, 1], [x, 2]⎦

Actual pure functions are a bit clunky. We need to declare them as lambda functions. A map operation would look like the following:

In [31]:
map(lambda k: k%5 == 0, range(10))

[True, False, False, False, False, True, False, False, False, False]

The ``map`` function is actually outdated. It returns a list, whereas list comprehensions return iterators. Iterators are essentially generating functions for list and they are very important to functional programming in Python. Iterators have a ``next()`` function to retrieve subsequent elements, making them very easy to loop over. For instance:

In [16]:
l = iter([1, 2, 3])
next(l)

1

This is, of course, not very useful, as we could have just used the list itself. List comprehensions are a more sensible way of getting iterators. Another way of creating an iterator is by defining a function that returns values through ``yield`` rather than through ``return``. This allows an internal state for the function and lets it continue where it left it off. For example:

In [17]:
def squares(N):
    for i in range(N):
        yield(i**2)

In [19]:
for j in squares(5):
    print(j)

0
1
4
9
16


There is no shortage of useful examples for using iterators. Take combinatoric functions, for instance:

In [24]:
import itertools
for combination in itertools.combinations([1, 2, 3, 4, 5], 2):
    print(combination)

(1, 2)
(1, 3)
(1, 4)
(1, 5)
(2, 3)
(2, 4)
(2, 5)
(3, 4)
(3, 5)
(4, 5)
